**STEP 1: The Work**

The goal of this 12-step program is to show you an enhanced library for string pattern matching, using the Python programming language doing some useful work.

First, let's review regular expressions which are already built into the Python programming langauge. The module is named "re", also refered as RE.

Say we are using regular expressions to analyze a text that contains simple sentences of the form:

* WHO went to WHERE looking for WHAT.

For example:

* I went to Louisiana, Texas and back looking for people and places.

Let's write a Python program using regular expressions that extracts all of these items, WHO, WHERE, and WHAT. This program will store this information in three variables and if we are successful, the result will be the following:

* who = 'I'
* where_to = ['Louisiana', 'Texas', 'back']
* what_for = ['people', 'places']

In [ ]:
import re # import the built-in regular expression module in Python
from pprint import PrettyPrinter # import the pretty-printer object
pp = PrettyPrinter(indent=2, width=160)
sentences = [ # Samples of text similar to those we want to analyze:
  "I went to Louisiana, Texas and back looking for people and places."
, "He went to Albertsons, Aldi, and Kroger looking for pineapples, and bananas."
, "You went to school and work looking for fun, frolic and fantasy."
]
for sentence in sentences:
    print(sentence)

**STEP 2: It's off to work we go.**

Let's try using the *re.search* function.

After a successful regular expression match, the *re.search* returns an *re.Match* object. To nicely display any Python object, the pretty-print function, *pprint*, is used. To gain access to any captured groups being returned, the *re.Match.groups* (or *re.Match.groupdict*) is used. The *groups* function returns a list of positionally captured groups and the *groupdict* function returns a dictionary of named captured groups.

This is going to be amazing! You mean all I have to do is create just one regular expression pattern to verify the validity of the text and simulataneously extract all the separate elements into structured data in *one fell swoop*?

In [ ]:
for sentence in sentences:
    if results := re.search(
          r"^(I|He|You)"                          # the caret denotes to match beginning of string
                                                  # (___), parenthesis, denotes to group and to capture
                                                  # _|_|_, the vertical-bar, denotes to match alternatives
          r" went to "                            # nothing special denotes to match this text literally
          r"([A-Za-z][a-z]*)"                     # [A-Za-z][a-z]* denotes to match a word, possibly capitalized
          r"(?:(?:,|,? and) ([A-Za-z][a-z]*))*"   # ___? denotes to match optionally, zero or one instances
                                                  # (___)* denotes matching a repetition of zero or more patterns
                                                  # (?:___) denotes to group but not to capture
                                                  # (?:(?:___) (___))*
                                                  #   comma or conjunction, or both
                                                  #   followed by a space
                                                  #   followed by a word possibly capitalized
          r" looking for "                        # match literal text
          r"([A-Za-z][a-z]*)"                     # group and capture the 1st thing
          r"(?:(?:,|,? and) ([A-Za-z][a-z]*))*"   # also capture all remaining things
                                                  # but ignore commas, spaces, and the word "and"
          r"\.$"                                  # the period character must be escaped with a backslash
                                                  # the dollar-sign denotes to match end of the string
        , sentence):
          pp.pprint(["Matched:", results.groups(), sentence])
    else: pp.pprint(["Unmatched!", sentence])

**STEP 3: What happened? I can't do my work.**

Where is Texas, Aldi, and frolic?\
Where are the two lists for the places and things?\
I think I might have choosen unwisely.\
\
Maybe there is a bug? Can it be that the construct (?:___) specifying a non-capturing group is in some way interferring with properly capturing its nested pattern? So let's modify these to simply be capturing groups instead.

In [ ]:
for sentence in sentences:
    if results := re.search(
          r"^(I|He|You)"
          r" went to"
          r" ([A-Za-z][a-z]*)"
          r"((?:,|,? and) ([A-Za-z][a-z]*))*" # (?:(?:___) (___))* becomes ((?:___) (___))*
          r" looking for"
          r" ([A-Za-z][a-z]*)"
          r"((?:,|,? and) ([A-Za-z][a-z]*))*" # (?:(?:___) (___))* becomes ((?:___) (___))*
          r"\.$"
        , sentence):
          pp.pprint(["Matched:", results.groups(), sentence])
    else: pp.pprint(["Unmatched!", sentence])

**STEP 4: What happened? I still can't do my work.**

Things seem worse than before.\
I think I might have choosen unwisely.\
\
Can it be that capturing elements of a reptition just isn't possible?\
Can it be that the *re.search* function will just not return a list of elements?\
Unfortunately, the RE module is working as designed and hence this limitation will likely never to be lifted. It will only ever just return the last element of the repitition, and will always refuse to capture the remaining parts.\
\
Unfortunately, the developers of the RE module might just have built this bug into the product as a feature. So, let's try one more time. Let's at least attempt to capture the entire text of these lists and not capture their nested elements.

In [ ]:
for sentence in sentences:
    if results := re.search(
          r"^(I|He|You)"
          r" went to"
          r" ([A-Za-z][a-z]*)"
          r"((?:(?:,|,? and) (?:[A-Za-z][a-z]*))*)" # ((?:___) (___))* becomes ((?:(?:___) (___))*)
          r" looking for"
          r" ([A-Za-z][a-z]*)"
          r"((?:(?:,|,? and) (?:[A-Za-z][a-z]*))*)" # ((?:___) (___))* becomes ((?:(?:___) (___))*)
          r"\.$"
        , sentence):
          pp.pprint(["Matched:", results.groups(), sentence])
    else: pp.pprint(["Unmatched!", sentence])

**STEP 5: What happens now? There is more work to be done.**

So, it appears this caliber of results is the best we can accomplish using the RE module, and that our initial desire to develop one regular expression pattern to extract structured data from text will not be fulfilled. For this task, it appears more coding will be necessary. Oh but I desperately wanted to avoid procedural coding all together. I wanted the pattern to look similar to the subject. I want the solution to resemble the problem.

Since we are limited to having just a single capture group to return a repitition in its entirety, and since individual elements can not be captured, let's at least merge the patterns for the first and remaining parts into one pattern for a proper comma-seperated string for later processing.

In [ ]:
for sentence in sentences:
    if results := re.search(
          r"^(I|He|You)"
          r" went to"
          r" ("
              r"(?:[A-Za-z][a-z]*)"
              r"(?:(?:,|,? and) (?:[A-Za-z][a-z]*))*"
          r")" # ((___)(?:(?:___) (___))*) becomes ((?:___)(?:(?:___) (___))*)
          r" looking for"
          r" ("
              r"(?:[A-Za-z][a-z]*)"
              r"(?:(?:,|,? and) (?:[A-Za-z][a-z]*))*"
          r")" # ((___)(?:(?:___) (___))*) becomes ((?:___)(?:(?:___) (___))*)
          r"\.$"
        , sentence):
          pp.pprint(["Matched:", results.groups(), sentence])
    else: pp.pprint(["Unmatched!", sentence])

**STEP 6: So now what happens? So much work. So little time.**

I must write more Python code! Ooh, somebody stop me.

In [ ]:
for sentence in sentences:
    if results := re.search(
          # (?P<name>___) denotes to match, group and capture by name
          r"^(?P<who>I|He|You)"
          r" went to"
          r" (?P<where>"
            r"(?:[A-Za-z][a-z]*)"
            r"(?:(?:,|,? and)? (?:[A-Za-z][a-z]*))*"
          r")" # (?P<where>___) denotes to capture group named where
          r" looking for"
          r" (?P<what>"
            r"(?:[A-Za-z][a-z]*)"
            r"(?:(?:,|,? and)? (?:[A-Za-z][a-z]*))*"
          r")" # 'who', 'where', and 'what' are keys of groupdict()
          r"\.$"
        , sentence):
          who = results.groupdict()['who']
          where_to = []
          for word_results in re.finditer(
              r"(?:(?:^|,? |,? and )(?!and)(?P<word>[A-Za-z][a-z]*))",
              results.groupdict()['where']
          ):  where_to.append(word_results.groupdict()['word'])
          what_for = []
          for word_results in re.finditer(
              r"(?:(?:^|,? |,? and )(?!and)(?P<word>[A-Za-z][a-z]*))",
              results.groupdict()['what']
          ):  what_for.append(word_results.groupdict()['word'])
          pp.pprint(["Matched:", who, where_to, what_for, sentence])
    else: pp.pprint(["Unmatched!", sentence])

**STEP 7: So what happened there? A beautiful mess.**

But it works. It validates the input text and produces three variables, one containing who, and two containing the list of where to and what for.

But is there another way?\
Let's try using the SNOBOL4python library instead.\
The following code will mount and import the SNOBOL4python package.

In [ ]:
!pip install SNOBOL4python==0.4.5
import sys
from pprint import pprint, pformat
## Thirty one (31) flavors of patterns to choose from ...
from SNOBOL4python import ε, σ, π, λ, Λ, ζ, θ, Θ, φ, Φ, α, ω
from SNOBOL4python import ABORT, ANY, ARB, ARBNO, BAL, BREAK, BREAKX, FAIL
from SNOBOL4python import FENCE, LEN, MARB, MARBNO, NOTANY, POS, REM, RPOS
from SNOBOL4python import RTAB, SPAN, SUCCEED, TAB
# Miscellaneous
from SNOBOL4python import GLOBALS, TRACE, PATTERN, Ϩ, STRING
from SNOBOL4python import ALPHABET, DIGITS, UCASE, LCASE, NULL
from SNOBOL4python import nPush, nInc, nPop, Shift, Reduce, Pop
GLOBALS(globals()) # Instantiate the global variable space

**STEP 8: What's happening? A new way. A new hope.**

To use the new PATTERN datatype provided by the SNOBOL4python Python module:

*   r"^" becomes *POS*(0)
*   r"$" becomes *RPOS*(0)
*   r"[a-z]" becomes *ANY*(LCASE)
*   r"xyz" becomes σ('xyz'), or alternatively
*   r"xyz" becomes σ('x') + σ('y') + σ('z')
*   r"x|z" becomes σ('x') | σ('z')
*   r"(\_\_\_)*" becomes *ARBNO*(___)
*   re.search(pattern, subject) becomes subject in PATTERN

Let's start by just getting the PATTERN to work, and not dealing with capturing any results.

In [ ]:
word = ANY(UCASE+LCASE) + (SPAN(LCASE) | ε())
delimiter = σ(', and ') | σ(' and ') | σ(', ')
for sentence in sentences:
    if sentence in \
          ( POS(0)
          + (σ('I') | σ('He') | σ('You'))
          + σ(' went to ')
          + word + ARBNO(delimiter + word)
          + σ(' looking for ')
          + word + ARBNO(delimiter + word)
          + σ('.')
          + RPOS(0)
          ):
          pp.pprint(['Matched.', sentence])
    else: pp.pprint(['Unmatched!', sentence])

**STEP 9: It's happening! My work is getting done.**

Now, let's decorate the above pattern with Python code to capture the PATTERN matching results into variables containing strings and lists.

* r"(?P\<name>\_\_\_) becomes ___ % "name"
* r"*no-can-do*" becomes λ("*python_code_string*")

In [ ]:
word = (ANY(UCASE+LCASE) + (SPAN(LCASE) | ε())) % "wrd"
delimiter = (σ(', and ') | σ(' and ') | σ(', '))
for sentence in sentences:
    if sentence in \
          ( POS(0)
          + (σ('I') | σ('He') | σ('You')) % "who"
          + σ(' went to ')     + word + λ("where_to = [wrd]")
          + ARBNO(delimiter    + word + λ("where_to.append(wrd)"))
          + σ(' looking for ') + word + λ("what_for = [wrd]")
          + ARBNO(delimiter    + word + λ("what_for.append(wrd)"))
          + σ('.')
          + RPOS(0)
          ):
          pp.pprint(['Matched:', who, where_to, what_for, sentence])
    else: pp.pprint(['Unmatched!', None, None, None, sentence])

**STEP 10: It happened! My work is done.**

Can it really be that easy? The solution resembles a description of the problem.

Using the theta function and immediate assignment operator in conjunction with the OUTPUT variable, you can trace the progress of the pattern matching scanner.

In [ ]:
word = Θ("OUTPUT") + (ANY(UCASE+LCASE) + (SPAN(LCASE) | ε())) @ "OUTPUT"
delimiter = Θ("OUTPUT") + (σ(', and ') | σ(' and ') | σ(', ')) @ "OUTPUT"
for sentence in sentences:
    if sentence in \
          ( Θ("OUTPUT") + POS(0)
          + Θ("OUTPUT") + (σ('I') | σ('He') | σ('You')) @ "OUTPUT"
          + Θ("OUTPUT") + σ(' went to ') @ "OUTPUT"
                         + word + ARBNO(delimiter + word)
          + Θ("OUTPUT") + σ(' looking for ') @ "OUTPUT"
                        + word + ARBNO(delimiter + word)
          + Θ("OUTPUT") + σ('.') @ "OUTPUT"
          + Θ("OUTPUT") + RPOS(0)
          ):
          print(f'Matched: {sentence}\n')
    else: print(f'Unmatched! {sentence}\n')

**STEP 11: It happened! My work is ensured.**

Now introducing the PATTERN phi, φ(r'___'). It will match a regular expression. And now a solution using regular expression patterns as an integral part of the new PATTERN datatype.

In [ ]:
word = φ(r'(?P<wrd>[A-Za-z][a-z]*)')
delimiter = φ(r'(?:,? and|,) ')
for sentence in sentences:
    if sentence in \
          ( φ(r'^')
          + (φ(r'(?P<who>I|He|You)'))
          + φ(r' went to ')     + word + λ("where_to = [wrd]")
          + ARBNO(delimiter     + word + λ("where_to.append(wrd)"))
          + φ(r' looking for ') + word + λ("what_for = [wrd]")
          + ARBNO(delimiter     + word + λ("what_for.append(wrd)"))
          + φ(r'\.$')
          ):
          pp.pprint(['Matched:', who, where_to, what_for, sentence])
    else: pp.pprint(['Unmatched!', None, None, None, sentence])

**STEP 12: What happens next. You can do any work you want!**

This SNOBOL4python module can process all four levels of the Chompsky heirarchy.\
This concludes this 12-step program.\
Now enjoy nirvana by seeing some real-world examples.

Let's process those TASA Treebank trees in the file from assignment #3, but let's first process those same sentences after being POS tagged by the UCREL CLAWS.

**STEP 12+1: Example 1: UCREL CLAWS POS-tagged sentences from assignment #3**

In [ ]:
CLAWS_5_in_TASA = """\
1_CRD :_PUN That_CJT the_AT0 power_NN1 of_PRF taxing_VVG it_PNP by_PRP the_AT0
states_NN2 may_VM0 be_VBI exercised_VVN so_AV0 as_AV0 to_TO0 destroy_VVI
it_PNP ,_PUN is_VBZ too_AV0 obvious_AJ0 to_TO0 be_VBI denied_VVN ._PUN
2_CRD :_PUN None_PNI ever_AV0 penned_VVD a_AT0 manifesto_NN1 as_AV0
stirring_AJ0 as_CJS the_AT0 one_PNI that_CJT appeared_VVD in_PRP the_AT0
first_ORD issue_NN1 of_PRF the_AT0 liberator_NN1 ,_PUN and_CJC no_AT0
other_AJ0 abolitionist_NN1 document_NN1 is_VBZ so_AV0 well_AV0 remembered_VVN
._PUN
"""

In [ ]:
claws_info = \
    ( POS(0)
    + λ("mem = dict()")
    + ARBNO(
        ( SPAN(DIGITS) % "num" + σ('_CRD :_PUN')
        + λ("num = int(num)")
        | (NOTANY("_") + BREAK("_")) % "wrd"
        + σ('_')
        + (ANY(UCASE) + SPAN(DIGITS+UCASE)) % "tag"
        + λ("if wrd not in mem:      mem[wrd] = dict()")
        + λ("if tag not in mem[wrd]: mem[wrd][tag] = 0")
        + λ("mem[wrd][tag] += 1")
        )
      + (σ(' \n') | σ(' ') | σ('\n'))
      )
    + RPOS(0)
    )
pp.pprint(claws_info)

In [ ]:
mem = None
if CLAWS_5_in_TASA in claws_info:
    pp.pprint(mem)
else: print("Boo!")

In [ ]:
{ ',': {'PUN': 2}, '.': {'PUN': 2},
  'None': {'PNI': 1}, 'That': {'CJT': 1},
  'a': {'AT0': 1}, 'abolitionist': {'NN1': 1}, 'and': {'CJC': 1},
  'appeared': {'VVD': 1}, 'as': {'AV0': 2, 'CJS': 1},
  'be': {'VBI': 2}, 'by': {'PRP': 1},
  'denied': {'VVN': 1}, 'destroy': {'VVI': 1}, 'document': {'NN1': 1},
  'ever': {'AV0': 1}, 'exercised': {'VVN': 1},
  'first': {'ORD': 1}, 'in': {'PRP': 1},
  'is': {'VBZ': 2}, 'issue': {'NN1': 1}, 'it': {'PNP': 2},
  'liberator': {'NN1': 1},
  'manifesto': {'NN1': 1}, 'may': {'VM0': 1},
  'no': {'AT0': 1},
  'obvious': {'AJ0': 1}, 'of': {'PRF': 2}, 'one': {'PNI': 1}, 'other': {'AJ0': 1},
  'penned': {'VVD': 1}, 'power': {'NN1': 1},
  'remembered': {'VVN': 1},
  'so': {'AV0': 2}, 'states': {'NN2': 1}, 'stirring': {'AJ0': 1},
  'taxing': {'VVG': 1}, 'that': {'CJT': 1}, 'the': {'AT0': 5}, 'to': {'TO0': 2},
  'too': {'AV0': 1},
  'well': {'AV0': 1}
}

In [ ]:
# This next example requires the file "CLAWS5inTASA.dat" on your Google Drive
# Skip this one step if you don't have the file.
from google.colab import drive
drive.mount('/content/modules', force_remount=True)
sys.path.append('/content/modules/My Drive/')

In [ ]:
mem = None
with open("/content/modules/My Drive/CLAWS5inTASA.dat", "r") as claws_file:
    lines = []
    while line := claws_file.readline():
        lines.append(line[0:-1])
    claws_data = ''.join(lines)
    if not claws_data in claws_info:
        print("Yikes")
    pp.pprint(mem)

**STEP 12+2: Example 2: Penn Treebank grammar trees from assignment #3**

In [ ]:
VBG_in_TASA = """\
(S (SBAR (IN that) (S (NP (NP (DT the) (NN power)) (PP (IN of) (S (VP (VBG
taxing) (NP (PRP it)) (PP (IN by) (NP (DT the) (NNS states))))))) (VP (MD
may) (VP (VB be) (VP (VBN exercised) (ADVP (RB so) (RB as)) (S (VP (TO to)
(VP (VB destroy) (NP (PRP it))))))))) (, ,)) (VP (VBZ is) (ADJP (RB too)
(JJ obvious)) (S (VP (TO to) (VP (VB be) (VP (VBN denied)))))) (.  .))

(S (S (NP (NN none)) (ADVP (RB ever)) (VP (VBN penned) (NP (DT a) (NN
manifesto)) (PP (IN as) (S (VP (VBG stirring) (PP (IN as) (NP (NP (DT the)
(NN one)) (SBAR (WHNP (WDT that)) (S (VP (VBD appeared) (PP (IN in) (NP (NP
(DT the) (JJ first) (NN issue)) (PP (IN of) (NP (DT the) (NN
liberator))))))))))))))) (, ,) (CC and) (S (NP (DT no) (JJ other) (NN
abolitionist) (NN document)) (VP (VBZ is) (ADVP (RB so) (RB well)) (VP (VBN
remembered)))) (.  .))

"""

In [ ]:
delim     = SPAN(" \n")
word      = NOTANY("( )\n") + BREAK("( )\n")
group     = σ('(') + word + ARBNO(delim + (ζ('group') | word)) + σ(')')
treebank  = POS(0) + ARBNO(ARBNO(group) + delim) + RPOS(0)
VBG_in_TASA in treebank # % "OUTPUT"

In [ ]:
def init_list(v): return λ(f"{v} = None; stack = []")
def push_list(v): return λ(f"stack.append(list()); stack[-1].append({v})")
def push_item(v): return λ(f"stack[-1].append({v})")
def pop_list():   return λ(f"stack[-2].append(tuple(stack.pop()))")
def pop_final(v): return λ(f"{v} = tuple(stack.pop())")
delim =           SPAN(" \n")
word =            NOTANY("( )\n") + BREAK("( )\n")
group =           ( σ('(')
                  + word % "tag"
                  + push_list("tag")
                  + ARBNO(delim + (ζ('group') | word % "wrd" + push_item("wrd")))
                  + pop_list()
                  + σ(')')
                  )
treebank =        ( POS(0)
                  + init_list("bank")
                  + push_list("'BANK'")
                  + ARBNO(push_list("'ROOT'") + ARBNO(group) + pop_list() + delim)
                  + pop_final("bank")
                  + RPOS(0)
                  )
pp.pprint([delim, word, group, treebank])

In [ ]:
bank = None
if VBG_in_TASA in treebank:
    pp.pprint(bank[1])
else: print("Boo!")

In [ ]:
# This next example requires the file "VBGinTASA.dat" on your Google Drive.
# Skip this one step if you don't have the file.

In [ ]:
bank = None
with open("/content/modules/My Drive/VBGinTASA.dat", "r") as bank_file:
    bank_source = bank_file.read()
    if bank_source in POS(0) + BAL() + RPOS(0):
        if bank_source in treebank:
            print(len(bank), "trees processed.")
    else: print("Boo!")